# How to add new statistics to the emulator

In this tutorial we'll go through how to add new statistics to the emulator package.

Let's start by loading the necessary packages

In [1]:
import numpy as np
import Emulator_functions as em_func

Here we need to set up some variables to be used later. This step should essentially be the same for training any new statistic (ideally this would all be already included within the emulator package, but this currently hasn't been implemented)

In [2]:
#get directory of the emulator pacckage
dir_em = em_func.__file__.split('/')[0:-1]
dir_em = '/'.join(dir_em)

#load latin hypercube, and random tests
L_cube = np.loadtxt(dir_em + '/Latin_hypercube_D6_N25_strength2_v2.txt')
tests = np.loadtxt(dir_em + '/random_cube_2.txt')

#set up variables about the simulations that have been ran
num_snaps=30
param_label = ['WDM mass, m_DM', 'A, fmin=A*fmax', 'Max stellar feedack efficiency, fmax','Star formation threshold, n^*_H','Stellar feedback transition scale, n_H0', 'Reionisation redshift, z']
snap_name='029_z000p000'


#initialise class to build new stats
em = em_func.emulator_build(param_label,L_cube,tests)

#get folder names for the training and test data
file_name,file_name_test=em.get_filename()

We now have everything we need to start processing the simulation to train new simulations. In this example let's actually train an existing statistic (specifically 'ApertureMeasurements_Mass_030kpc_PartType4') but treat this as if it's a new one.

First we need to load both the training and testing data

Don't worry about the naming structure here, the emulator class will take care of all of this under the hood, and we're only doing this here to pretend like this is new data.

In [3]:
training_data = np.load(dir_em+'/Training_data/ApertureMeasurements_Mass_030kpc_PartType4029.pickle', allow_pickle=True)
print(training_data.shape)
print(training_data)

(25, 3)
[[ 0.53490553  0.62876284  0.88337681]
 [ 0.50581358  0.5934641   0.81270928]
 [ 0.57042299  0.62934229  0.77134405]
 [ 0.47293812  0.33474834  0.70520603]
 [ 0.49663486  0.29996792  0.77520863]
 [ 0.68173858  0.78074359  0.96876891]
 [ 0.28290461  0.34669911  0.52887278]
 [-0.78924188 -0.73066947 -0.48149397]
 [-1.2551146  -1.21108923 -0.96534417]
 [ 0.76658417  0.82473764  1.03260688]
 [-0.11519599 -0.11635008  0.14113005]
 [-0.72685034 -0.69762891 -0.50774703]
 [-1.04343357 -1.00298511 -0.84205055]
 [ 0.68570185  0.7859519   1.00553953]
 [ 0.0799317   0.22386317  0.38063946]
 [-0.58373603 -0.53592414 -0.38751062]
 [-1.51943184 -1.48142449 -1.28413752]
 [ 0.57633525  0.70700925  0.91918822]
 [ 0.44789851  0.51968266  0.7729039 ]
 [-0.16552273 -0.11772716  0.05311033]
 [-1.27904966 -1.16399147 -0.98829113]
 [ 0.58462522  0.68515714  0.9299305 ]
 [ 0.34436731  0.41370366  0.69221729]
 [-0.66307548 -0.59355079 -0.43213513]
 [-0.94570286 -0.87436404 -0.66997169]]


Here we are going to treat this as the training data. As we can see the structure is a 2D array of (25,3). This represents the 25 training simulations measured for the 3 haloes. In this particular example this represents the stellar mass of the host today.

In general the training data must be an N-dimensional array of the form (25,...). Where the extra dimensions can be as complex or as simple as you like. It is also important that the 1st dimension (i.e. the 25) is ordered to match that of the latin hypercube. This ordering is given by `L_cube` or `em.get_filename()`.

For your own analysis this step will need to be replaced by whatever statistics you are interested in. Though the steps of training the emulator should be identical.

Let's now go and train this as a new statistic.

In [4]:
desc = 'Stellar mass within 30 kpc at z=0'
halos=['halo_61','halo_32','halo_04']
em.train(training_data,'029',x = halos, statistic = 'Stellar_mass_30kpc',description = desc,train_seperataely=True, replace = True)

Statistic exists, overwriting!


/home/pflt47/.local/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/pflt47/.local/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 5 of parameter k1__k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/pflt47/.local/lib/python3.11/site-packages/sklearn/gaussian_process/kernels.py:430: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
/home/pflt47/.local/lib/python3.11/site-packages/sklearn/g

This step will likely spit out a few warnings. Generally these are nothing to worry about, and often happen when one (or more) of the parameters don't have much impact, and therefore there is no clearly defined 'correct' choice for the hyper parameters.

Let's go through the key inputs to this function:

	- training_data: must be a (25,...) array, ordered the same as the latin hypercube
	- tag: This is the snapshot number of the data you're providing. Here it was the present day snapshot, so we provided '029'

kwargs

	- x: this is the 'x' axis of the data, if there is one. This is not used by the emulator so can in principle be anything you like (both lists, strings, floats, etc), and can be useful to the end user. Here we provided the halo numbers.
	- statistic: This is the name you provide to the statistic you are training. This must be a string. This is also the same phrase that will be used later to call this statistic, so try to use something relatively short and clear.
	- `description`: A description of the statistic. Again, something not used by the emulator but useful for the end user
	- train_seperately: This should be a boolean. By default it is set to False, and specified if the hyperparameters should be independently or jointly constrained. Generally I would recommend this to be set to True
	- replace: boolean to specify if an existing statistic should be overwritten or not. Default set to False. Be careful here not to delete existing statistics you want to keep.


Let's now make sure we can call this as a new statistic.

In [5]:
em_pred = em_func.emulator()

fiducial = np.array([np.infty,0.1,np.log10(3),np.log10(0.1),np.log10(50),11.5]).reshape(1,-1)

stel_mas, z = em_pred.predict('Stellar_mass_30kpc',fiducial,0.0,normalised=False)

print(stel_mas)

Sampling redshift 0.00
[0.2846     0.28034367 0.50654545]


Great, that statistic works and is now part of the existing sample. The values should also match very closely those in the `Emulator_first_use` notebook (though likely there will be slight differences in the values).

This should hopefully be everything you need to start adding new statistics yourself. The final thing I would strongly recommend is testing the accuracy of the emulator on the test data. In this notebook these are the `tests` parameters (in normalised coordinates) and the simulations in the  `file_name_test` folders. This will let you evaluate if the emulator is actually doing a good job or not.

